# LangGraph Mini-Lab 2 — Looping Graph

**Author:** Dr. Dasha Trofimova

Loop until a stopping condition is met.

In [ ]:
!pip install -q langgraph

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

class S(TypedDict):
    text: str
    target_len: int
    steps: int

def summarize_once(s: S) -> S:
    txt = s["text"].strip()
    if len(txt) <= s["target_len"]:
        return {"text": txt, "steps": s["steps"]}
    parts = [p for p in txt.split(".") if p.strip()]
    if len(parts) <= 1:
        txt2 = txt[:s["target_len"]].rstrip() + "..."
    else:
        txt2 = ". ".join(parts[:-1]).strip() + "."
    return {"text": txt2, "steps": s["steps"] + 1}

def should_continue(s: S):
    return len(s["text"]) > s["target_len"]

g = StateGraph(S)
g.add_node("shrink", summarize_once)
g.add_conditional_edges("shrink", should_continue, {True: "shrink", False: END})
g.set_entry_point("shrink")
app = g.compile()

state = {"text": "This is a long paragraph. It has multiple sentences. We will shorten it iteratively.", "target_len": 45, "steps": 0}
out = app.invoke(state)
print(out)

### 🔁 Graph Visualization
Loop until the text length ≤ target_len.

In [ ]:
!apt-get -qq update && apt-get -qq install -y graphviz > /dev/null
!pip install -q graphviz

In [ ]:
from graphviz import Digraph
gdot = Digraph(comment="Looping Graph", format="png")
gdot.attr(rankdir="LR", bgcolor="white")
gdot.node("shrink", "shrink()\n(summarize_once)", shape="box", style="rounded,filled", fillcolor="#FEF3C7")
gdot.node("end", "END", shape="doublecircle", style="filled", fillcolor="#F3F4F6")
gdot.edge("shrink", "shrink", label="len(text) > target_len")
gdot.edge("shrink", "end", label="else")
display(gdot)

Run the previous cell to see how many **steps** were taken before reaching `END`.


## 🔁 Agent Anatomy — Looping Graph

**What this "agent" does:**  
Repeatedly shortens `text` until its length ≤ `target_len` (toy summarizer) using a loop edge.

**State schema:**  
- `text` *(str)* → current working text  
- `target_len` *(int)* → maximum desired length  
- `steps` *(int)* → number of loop iterations

**Nodes:**  
- **shrink** → removes the last sentence or trims to target; increments `steps`.  
- **END** → stops when `len(text) ≤ target_len`.

We add descriptions and print a short trace per iteration.


In [ ]:

from langgraph.graph import StateGraph, END
from typing import TypedDict

class S(TypedDict):
    text: str
    target_len: int
    steps: int

node_desc = {
    "shrink": "Summarizer: if text is longer than target_len, drop the last sentence or trim."
}

def trace(name):
    def deco(fn):
        def wrapped(s: S):
            print(f"\n▶ Node: {name} — {node_desc[name]}")
            before_len = len(s["text"])
            out = fn(s)
            after_len = len(out["text"])
            print(f"   Step {out['steps']}: length {before_len} → {after_len}")
            return out
        wrapped.__doc__ = node_desc[name]
        return wrapped
    return deco

@trace("shrink")
def summarize_once(s: S) -> S:
    txt = s["text"].strip()
    if len(txt) <= s["target_len"]:
        return {"text": txt, "target_len": s["target_len"], "steps": s["steps"]}
    parts = [p for p in txt.split(".") if p.strip()]
    if len(parts) <= 1:
        txt2 = txt[:s["target_len"]].rstrip() + "..."
    else:
        txt2 = ". ".join(parts[:-1]).strip() + "."
    return {"text": txt2, "target_len": s["target_len"], "steps": s["steps"] + 1}

def should_continue(s: S):
    return len(s["text"]) > s["target_len"]

g = StateGraph(S)
g.add_node("shrink", summarize_once)
g.add_conditional_edges("shrink", should_continue, {True: "shrink", False: END})
g.set_entry_point("shrink")
app = g.compile()

state = {
    "text": "This is a long paragraph. It has multiple sentences. We will shorten it iteratively so it fits on one slide.",
    "target_len": 60,
    "steps": 0
}
print("\n=== Trace: looping until condition ===")
out = app.invoke(state)
print("\nFinal state:", out)
